## Problem Statement - Sentiment Analysis of movie review dataset

Dataset location : http://www.cs.cornell.edu/people/pabo/movie-review-data/



## Importing Libraries

In [1]:
import numpy as np
import pandas as pd

## Import Dataset

In [3]:
df = pd.read_csv('../datasets/TextFiles/moviereviews.tsv',sep='\t')

df.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [5]:
len(df)

2000

In [6]:
df['review'][0]

'how do films like mouse hunt get into theatres ? \r\nisn\'t there a law or something ? \r\nthis diabolical load of claptrap from steven speilberg\'s dreamworks studio is hollywood family fare at its deadly worst . \r\nmouse hunt takes the bare threads of a plot and tries to prop it up with overacting and flat-out stupid slapstick that makes comedies like jingle all the way look decent by comparison . \r\nwriter adam rifkin and director gore verbinski are the names chiefly responsible for this swill . \r\nthe plot , for what its worth , concerns two brothers ( nathan lane and an appalling lee evens ) who inherit a poorly run string factory and a seemingly worthless house from their eccentric father . \r\ndeciding to check out the long-abandoned house , they soon learn that it\'s worth a fortune and set about selling it in auction to the highest bidder . \r\nbut battling them at every turn is a very smart mouse , happy with his run-down little abode and wanting it to stay that way . \r\

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
label     2000 non-null object
review    1965 non-null object
dtypes: object(2)
memory usage: 31.4+ KB


## Missing Values

In [8]:
df.isnull().sum()

label      0
review    35
dtype: int64

## Dropping missing values

In [9]:
df.dropna(inplace=True)

In [10]:
df.isnull().sum()

label     0
review    0
dtype: int64

In [11]:
len(df)

1965

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1965 entries, 0 to 1999
Data columns (total 2 columns):
label     1965 non-null object
review    1965 non-null object
dtypes: object(2)
memory usage: 46.1+ KB


## Detecting empty strings and removing them

In [13]:
blanks = []

for index, label, review in df.itertuples():
    if type(review)==str:
        if review.isspace():
            blanks.append(index)
            
print(len(blanks),'blanks:',blanks)

27 blanks: [57, 71, 147, 151, 283, 307, 313, 323, 343, 351, 427, 501, 633, 675, 815, 851, 977, 1079, 1299, 1455, 1493, 1525, 1531, 1763, 1851, 1905, 1993]


In [14]:
df2 = pd.DataFrame({'a':[1,2,3,4],'b':[5,6,7,8],'c':[9,10,11,12]})

df2.head()

,a,b,c
0,1,5,9
1,2,6,10
2,3,7,11
3,4,8,12


In [ ]:

for r in df2.itertuples():
    print(r)

In [ ]:
for t in df[:10].itertuples():
  print(t)

In [ ]:
df.drop(blanks,inplace=True)
len(df)

## Taking a look at label column

In [15]:
df['label'].value_counts()

neg    983
pos    982
Name: label, dtype: int64

## Split train and test datasets

In [16]:
from sklearn.model_selection import train_test_split

X = df['review']
y=df['label']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

## Bulid a pipeline to vectorize the data and then train and fit into model

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [19]:
'''apply vectorize
clf = MultinomialNB()
clf.fit'''

# Naive Bayes
NBClx = Pipeline([('tfidf',TfidfVectorizer()),
                  ('clf',MultinomialNB())])

# Linear SVC
SVCClx = Pipeline([('tfidf',TfidfVectorizer()),
                  ('clf',LinearSVC())])

## Training the classifier

In [21]:
NBClx.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

## Run predictions and analyse the results

In [22]:
pred = NBClx.predict(X_test)

In [23]:
print('confusion matrix : \n',confusion_matrix(y_test,pred))

confusion matrix : 
 [[263  33]
 [ 83 211]]


In [24]:
print('classification report : \n',classification_report(y_test,pred))

classification report : 
               precision    recall  f1-score   support

         neg       0.76      0.89      0.82       296
         pos       0.86      0.72      0.78       294

    accuracy                           0.80       590
   macro avg       0.81      0.80      0.80       590
weighted avg       0.81      0.80      0.80       590



In [26]:
print('accuracy score : \n',accuracy_score(y_test,pred))
asc = accuracy_score(y_test,pred)

accuracy score : 
 0.8033898305084746


## Looking at the SVC pipeline

In [27]:
SVCClx.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [28]:
pred = SVCClx.predict(X_test)

In [29]:
print('Confusion Matrix : \n',confusion_matrix(y_test,pred))

Confusion Matrix : 
 [[254  42]
 [ 53 241]]


In [30]:
print('accuracy score : \n',accuracy_score(y_test,pred))
asc2 = accuracy_score(y_test,pred)

accuracy score : 
 0.8389830508474576


In [33]:
myReview = "A movie I really wanted to see. But the storyline was terrible. I assume the producers had different intention but it was lacking coherence."

myReview

'A movie I really wanted to see. But the storyline was terrible. I assume the producers had different intention but it was lacking coherence.'

In [34]:
print(NBClx.predict([myReview]))

['neg']


In [35]:
print(SVCClx.predict([myReview]))

['neg']


In [37]:
myReview = "Great movie, I really loved the way something complex like space time physics was linked to spirituality and love"

myReview

'Great movie, I really loved the way something complex like space time physics was linked to spirituality and love'

In [38]:
print(NBClx.predict([myReview]))

['pos']


In [39]:
print(SVCClx.predict([myReview]))

['pos']


## Great Job !